In [1]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Using TensorFlow backend.


In [2]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [3]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [4]:
from keras.layers import Input
input_img = Input(shape = (32, 32, 3))

In [5]:
from keras.layers import Conv2D, MaxPooling2D
tower_1 = Conv2D(64, (1,1), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(64, (3,3), padding='same', activation='relu')(tower_1)
tower_2 = Conv2D(64, (1,1), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(64, (5,5), padding='same', activation='relu')(tower_2)
tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_img)
tower_3 = Conv2D(64, (1,1), padding='same', activation='relu')(tower_3)

In [8]:
import keras
output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis = 3)

In [9]:
from keras.layers import Flatten, Dense
output = Flatten()(output)
out    = Dense(10, activation='softmax')(output)

In [10]:
from keras.models import Model
model = Model(inputs = input_img, outputs = out)
# print model.summary()

In [12]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [ ]:
from keras.optimizers import SGD
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 91s - loss: 0.9521 - acc: 0.6686 - val_loss: 1.1217 - val_acc: 0.6161
Epoch 2/25
50000/50000 [==============================] - 90s - loss: 0.7569 - acc: 0.7381 - val_loss: 1.0909 - val_acc: 0.6408
Epoch 3/25
50000/50000 [==============================] - 91s - loss: 0.6247 - acc: 0.7852 - val_loss: 1.1519 - val_acc: 0.6284
Epoch 4/25
50000/50000 [==============================] - 92s - loss: 0.5258 - acc: 0.8202 - val_loss: 1.1982 - val_acc: 0.6303
Epoch 5/25
50000/50000 [==============================] - 92s - loss: 0.4415 - acc: 0.8500 - val_loss: 1.2372 - val_acc: 0.6391
Epoch 6/25
50000/50000 [==============================] - 92s - loss: 0.3700 - acc: 0.8770 - val_loss: 1.3155 - val_acc: 0.6384
Epoch 7/25
50000/50000 [==============================] - 92s - loss: 0.3095 - acc: 0.9009 - val_loss: 1.3880 - val_acc: 0.6296
Epoch 8/25
50000/50000 [==============================

In [ ]:
from keras.models import model_from_json
import os
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights(os.path.join(os.getcwd(), 'model.h5'))

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))